In [4]:
# Import Libraries 
import hakom
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import getpass
import urllib.request, json 
import pandas as pd
from datetime import datetime, timedelta
#teständerung

In [25]:
api_pwd = getpass.getpass()


In [26]:
auth_params = hakom.authorization.AuthParams(
    username = 'aeneas',
    password = api_pwd,
)

service_config = hakom.service.ServiceConfig(
    base_url="https://fehostedwebapp.azurewebsites.net/MAIN/api/",
    auth_params=auth_params,
    repository='ZAMS'
)

hakom.initialize(service_config)

In [5]:
def parse_inmet(station_str):
    data =[]

    for year in range(2018,2023):

        
        date_begin = '{}-01-01'.format(year)
        date_end = '{}-12-31'.format(year)

        url_str = 'https://apitempo.inmet.gov.br/estacao/{}/{}/{}'.format(date_begin, date_end, str(station_str))
        print(url_str)


        # Edit: add loop to run over list of selected stations and dates

        with urllib.request.urlopen(url_str) as url:
            temp = pd.read_json(url.read().decode())
        data.append(temp)
    data = pd.concat(data)

    return data


In [7]:
data = parse_inmet('A521')
data.shape

https://apitempo.inmet.gov.br/estacao/2018-01-01/2018-12-31/A521
https://apitempo.inmet.gov.br/estacao/2019-01-01/2019-12-31/A521
https://apitempo.inmet.gov.br/estacao/2020-01-01/2020-12-31/A521
https://apitempo.inmet.gov.br/estacao/2021-01-01/2021-12-31/A521
https://apitempo.inmet.gov.br/estacao/2022-01-01/2022-12-31/A521


(43824, 27)

In [8]:
data.shape

(43824, 27)

In [12]:
def timeframe_inmet(data):
    data['timestamp'] = pd.to_datetime(data['DT_MEDICAO']) + pd.to_timedelta(data['HR_MEDICAO'] // 100, unit='h')
    data = data.set_index('timestamp')
    data = data.asfreq('h')
    
    df_temp = data[['TEM_INS']]
    df_wdir = data[['VEN_DIR']]
    df_wspeed = data[['VEN_VEL']]
    return data, df_temp


In [13]:
df,df_temp = timeframe_inmet(data)


In [20]:
def inmet_TEMP_into_TSM_format(df):
    df = hakom.pandas_interop.create_timestamp_colum_from_index(df)
    df['flag'] = 9
    one_hour_interval = hakom.types.Interval(hakom.types.Intervals.Hour)
    unit_label = '°C'
    tsdc = hakom.pandas_interop.to_hakom_tsdata_collection(
        df.dropna(),
        interval=one_hour_interval,
        unit=unit_label,
        dt_flag_value_order=[2,1,3]
    )
    return tsdc

In [21]:
df_tsm = inmet_TEMP_into_TSM_format(df_temp)
df_tsm.Data[:3]

[TimeSeriesDataItemTimeStamp(From='2018-01-01T00:00:00.000000Z', Value=21.7, Flag=9),
 TimeSeriesDataItemTimeStamp(From='2018-01-01T01:00:00.000000Z', Value=21.3, Flag=9),
 TimeSeriesDataItemTimeStamp(From='2018-01-01T02:00:00.000000Z', Value=21.7, Flag=9)]

In [24]:
def create_TSM_timeseries(time_series_name, unit_label, interval = 'one_hour_interval'):
     definition = hakom.types.TimeSeriesDefinition(
     Name=time_series_name,
     Type=hakom.types.TimeSeriesTypes.Begin,
     Unit=unit_label,
     Interval=interval,
    )
     created = hakom.repositories.timeseries.create(definition)
     return created

In [27]:
a = create_TSM_timeseries('aeneas_test_again', '°C')

BadRequestError: Could not deserialize body. - Details: The JSON value could not be converted to HAKOM.Framework.Components2016.TimeSeries.Interval. Path: $.Interval | LineNumber: 0 | BytePositionInLine: 67. Path: $ | LineNumber: 0 | BytePositionInLine: 1. Body: {"Name":"aeneas_test_again","Type":1,"Interval":"one_hour_interval","Unit":"°C","PreserveDuplicatedValues":false,"Quotation":false,"Audit":false,"Compressed":false}

In [28]:
test_time_series_name = 'temperature_test_aeneas'

ts_def = hakom.repositories.timeseries.get(test_time_series_name)
created = ts_def['ID']
ts_def

{'ID': 2500,
 'Name': 'temperature_test_aeneas',
 'Description': '',
 'Type': 1,
 'Interval': {'Value': 3, 'Multiplier': 1},
 'Unit': '°C',
 'Table': None,
 'ArchiveTable': None,
 'Formula': None,
 'TimeZone': None,
 'StartOfYear': 1,
 'StartOfDay': 0,
 'PreserveDuplicatedValues': False,
 'Aggregation': None,
 'QuotationDefaultBehavior': None,
 'Quotation': False,
 'Audit': False,
 'Compressed': False,
 'ExchangeTimeSeries': []}

In [29]:
hakom.repositories.timeseries.data.save(created, df_tsm)

True

In [33]:
ts_def = hakom.repositories.timeseries.get(2516)

In [32]:
min_date = df_temp.iloc[0]['from']
max_date = df_temp.iloc[-1]['from']
data_query = {"from": min_date, "to": max_date}
retrieved_data = hakom.repositories.timeseries.data.get(created, data_query)

ret_df = hakom.pandas_interop.to_pandas_dataframe(retrieved_data)
ret_df.plot(y=['Value'])

KeyError: 'from'